In [ ]:
pip install pandas openpyxl

In [ ]:
import pandas as pd
import math as m
import matplotlib.pyplot as plt
import numpy as np
from openpyxl import load_workbook

In [ ]:
pip install pandas xlrd openpyxl

Здесь импортируем все таблички, которые нам нужны для анализа работы каждого водителя. Пока что я импортирую только заправку и торгово-транспортные накладные

In [ ]:
objets_transport_docs = pd.read_excel("товарно-транспортные накладные.xls", index_col=7)
zapravka = pd.read_excel("заправка 01.09.2023-30.09.2023.xls", index_col=4)
tc = pd.read_excel("затраты на тс 01.09.2023-30.09.2023.xls", skiprows=3, engine='xlrd')
platon = pd.read_excel("платон.xls")
avtohodki = pd.read_excel('автоходки за период 01.09.2023-30.09.2023.xls',skiprows=3)

Дальше идут функции, которые используются в коде. Их нужно просто прощелкать, иначе не заработает.

In [ ]:
def datatime_split(df, start_data, end_data):

  df['Дата'] = df['Дата'].astype(str)
  df[['Дата','Время']] = df['Дата'].str.split(' ', 1, expand=True)
  df['Дата'] = pd.to_datetime(df['Дата'])
  df['Время'] = pd.to_datetime(df['Время'], format='%H:%M:%S').dt.time
  df_sorted = df.loc[(df['Дата'] >= start_data) & (df['Дата'] <= end_data)]
  return df_sorted

In [ ]:
def driver_split(df, name):
  df['Водитель'] = df['Водитель'].astype(str)
  filtered_df = df.loc[df['Водитель'].str.contains(name)]
  return filtered_df

In [ ]:
def checking_split(df):
  df['Проверен'] = df['Проверен'].astype(str)
  filtered_df = df.loc[df['Проверен'].str.contains('Да')]
  return filtered_df

In [ ]:
def procent(df):
  word2find = 'гр. процент'
  return df[~df['Номенклатура'].str.contains(word2find)]

In [ ]:
def general_parameters(df, parameter):
  df[parameter] = df[parameter].astype(float)
  return df[parameter].sum()

In [ ]:
def general_comments(df, parameter):
  df[parameter] = df[parameter].astype(str)
  df[parameter] = df[parameter].fillna(' ')
  return ' '.join(df[parameter].apply(str))

In [ ]:
def comments(df, parameter):
  df[parameter] = df[parameter].astype(str)
  return df[parameter]

In [ ]:
def split(df, column_name, element):
  if column_name not in df.columns:
    return 'no'

  else:
    df[column_name] = df[column_name].astype(str)
    filtered_df = df.loc[df[column_name].str.contains(element)]
    return filtered_df

In [ ]:
def unique(df, column_name):
  if column_name not in df.columns:
    return 'no'

  else:
    filtered_df = df[column_name].unique()
    return filtered_df

Здесь нужно ввести две даты начала и конца периода в формате дд-мм-гггг

In [ ]:
start_data = input()
end_data = input()

01-09-2023
30-09-2023


Здесь введенные массивы из 1С, и они фильтруются по выбранной дате

In [ ]:
objets_transport_docs = procent(datatime_split(pd.read_excel("товарно-транспортные накладные.xls"), start_data, end_data))
zapravka = datatime_split(pd.read_excel("заправка 01.09.2023-30.09.2023.xls"), start_data, end_data)
drivers = unique(objets_transport_docs,'Водитель')

<ipython-input-342-dd9f32dcddba>:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['Дата','Время']] = df['Дата'].str.split(' ', 1, expand=True)
<ipython-input-342-dd9f32dcddba>:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Дата'] = pd.to_datetime(df['Дата'])
<ipython-input-342-dd9f32dcddba>:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_sorted = df.loc[(df['Дата'] >= start_data) & (df['Дата'] <= end_data)]
<ipython-input-342-dd9f32dcddba>:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['Дата','Время']] 

Здесь сам цикл. Он обрабатывает имеющиеся таблички и  выдает сформированную таблицу по всем заявленным в ТТН водителям. Можно по любому списку водителей/по машинам/по чему угодно

In [ ]:
cars_without_cost = 0
cars_without_toplivo = 0
print('Всего водителей в указанный период: ',len(drivers))
data = {'Водитель': [0], 'Машина' :[0], 'Километраж' : [0],'Расход топлива (л)' : [0], 'Расход топлива (рубли)' : [0], 'Обслуживание ТС' : [0], 'Суммарная стоимость доставок' : [0], 'Платон' : [0]}
df = pd.DataFrame(data)

data1 = {'Водитель': [0], 'Километраж' : [0],'Расход топлива (л)' : [0], 'Расход топлива (рубли)' : [0], 'Обслуживание ТС' : [0], 'Суммарная стоимость доставок' : [0], 'Платон' : [0]}
df1 = pd.DataFrame(data1)

for j in range(len(drivers)):
  total_tc = 0
  name = drivers[j]
  data_driver_ttn = checking_split(driver_split(objets_transport_docs, name))
  data_driver_zapravka = driver_split(zapravka, name)
  data_driver_avtohodki = driver_split(avtohodki, name)

  param_zapravka = ['Дата','Топливо','Автомобиль','Заправка на стороне','Цена','Количество','Сумма']
  param_ttn = ['Дата','Покупатель','Автомобиль','Количество ЗАВОД','Количество КЛИЕНТУ','Стоимость доставки']

  car_names = data_driver_zapravka['Автомобиль'].unique()
  total_volume = general_parameters(data_driver_zapravka, 'Количество')
  total_sum = general_parameters(data_driver_zapravka,'Сумма')
  total_cost = general_parameters(data_driver_ttn, 'Стоимость доставки')
  total_comment = general_comments(data_driver_ttn, 'Комментарий')
  total_avtohodki = general_comments(data_driver_avtohodki, 'км')
  if total_volume == 0.0:
      cars_without_toplivo = 1 + cars_without_toplivo
  if total_cost == 0.0:
      cars_without_cost = 1 + cars_without_cost

  for i in range(len(car_names)):
    tc_percar = general_parameters(split(tc, 'ТС', str(car_names[i])), 'Сумма Расход')
    platon_percar = general_parameters(split(platon, 'Транспортное средство', str(car_names[i])), 'Сумма')
    avtohodki_percar = general_parameters(split(avtohodki, 'Авто', str(car_names[i])), 'км')
    total_tc = total_tc + tc_percar
    volume_percar = general_parameters(split(data_driver_zapravka, 'Автомобиль', str(car_names[i])), 'Количество')
    cost_percar = general_parameters(split(data_driver_zapravka, 'Автомобиль', str(car_names[i])), 'Сумма')
    dostavka_percar = general_parameters(split(data_driver_ttn, 'Автомобиль', str(car_names[i])), 'Стоимость доставки')
    new_data = {'Водитель': name, 'Машина' : car_names[i], 'Километраж' : avtohodki_percar,'Расход топлива (л)' : volume_percar, 'Расход топлива (рубли)' : cost_percar, 'Обслуживание ТС' : tc_percar, 'Суммарная стоимость доставок' : dostavka_percar, 'Платон' : platon_percar}
    df = df.append(new_data, ignore_index=True)

  new_data1 = {'Водитель': name,  'Километраж' : total_avtohodki,'Расход топлива (л)' : total_volume, 'Расход топлива (рубли)' : total_sum, 'Обслуживание ТС' : total_tc, 'Суммарная стоимость доставок' : total_cost}
  df1 = df1.append(new_data, ignore_index=True)

Всего водителей в указанный период:  48


<ipython-input-344-d24287829c1b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Проверен'] = df['Проверен'].astype(str)
<ipython-input-347-aee5e235e976>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[parameter] = df[parameter].fillna(' ')
<ipython-input-349-b4bd58b8a48e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

df массив - массив по отдельным машинам, отсортированным по одному водителю

df1 - массив по водителям, все данные по машинам суммируются

In [ ]:
df

,Водитель,Машина,Километраж,Расход топлива (л),Расход топлива (рубли),Обслуживание ТС,Суммарная стоимость доставок,Платон
0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
1,Уразов Санжар Солехович,В551ВО790 МАН 1932 TGS 19.400 4х2 BLS-WW,15361.31,30186.06,1551415.33,493851.95,486486.00,26600.10
2,Уразов Санжар Солехович,Т764МВ750 АБС БСК 6775-50,4439.24,1065.00,47974.10,171382.78,0.00,1997.16
3,Уразов Мизроб Солехович,О030ХЕ123 МАН,15153.47,27747.45,1418818.83,525699.25,483598.00,26426.63
4,Уразов Мизроб Солехович,Т764МВ750 АБС БСК 6775-50,4439.24,700.00,27986.00,171382.78,0.00,1997.16
5,Рашидов Фаёзбек Фарходович,Х194ВА797 КАМАЗ,15069.15,29342.20,1497242.52,843349.08,451930.00,25629.88
6,Султонов Ихтиёр Бахтиёрович,У162ХК 750 МАН TGS 19.400 4х2 BLS-WW,13759.12,27453.63,1393026.24,375724.68,407950.00,0.00
7,Рузиев Илхом Иброхимович,В247ХО750 МАN TGA 18.390 4х2 BLS,12423.28,25522.00,1262050.32,462719.39,374602.00,21416.71
8,Азимов Тулкин Толибович,М991ХС750 МАН 1932 TGS 19.400 4х2 BLS-WW,11087.25,31343.81,1569388.61,717059.65,490921.00,17884.18
9,Бешимов Мадамиджон Салимбой Угли,О327ВН790 МАН TGA 18.480 4х2 BLS,7902.93,6291.00,358525.25,360243.61,56256.00,12873.55


In [ ]:
df1

,Водитель,Километраж,Расход топлива (л),Расход топлива (рубли),Обслуживание ТС,Суммарная стоимость доставок,Платон,Машина
0,0,0.00,0.00,0.00,0.00,0.00,0.00,NaN
1,Уразов Санжар Солехович,4439.24,1065.00,47974.10,171382.78,0.00,1997.16,Т764МВ750 АБС БСК 6775-50
2,Уразов Мизроб Солехович,4439.24,700.00,27986.00,171382.78,0.00,1997.16,Т764МВ750 АБС БСК 6775-50
3,Рашидов Фаёзбек Фарходович,15069.15,29342.20,1497242.52,843349.08,451930.00,25629.88,Х194ВА797 КАМАЗ
4,Султонов Ихтиёр Бахтиёрович,13759.12,27453.63,1393026.24,375724.68,407950.00,0.00,У162ХК 750 МАН TGS 19.400 4х2 BLS-WW
5,Рузиев Илхом Иброхимович,12423.28,25522.00,1262050.32,462719.39,374602.00,21416.71,В247ХО750 МАN TGA 18.390 4х2 BLS
6,Азимов Тулкин Толибович,11087.25,31343.81,1569388.61,717059.65,490921.00,17884.18,М991ХС750 МАН 1932 TGS 19.400 4х2 BLS-WW
7,Бешимов Мадамиджон Салимбой Угли,3871.44,1114.00,55901.49,464662.39,0.00,1434.21,К039ВХ977 58149Y-01 Автобетоносмеситель на шас...
8,Саломов Шухрат Рахмонкулович,3365.22,7386.00,330726.82,138937.86,0.00,0.00,А004ВН50 МАН TGA 33.350 6х4 88-WW
9,Джураев Акмал Икромович,6039.88,13155.00,696676.49,965826.50,453227.91,3082.92,Х240ВА797 КАМАЗ


Здесь вводится имя работника, который интересует и все массивы фильтруются только под этого работника

In [ ]:
name = drivers[0]
data_driver_ttn = checking_split(driver_split(objets_transport_docs, name))
data_driver_zapravka = driver_split(zapravka, name)

Халимов Феруз Косимович


<ipython-input-177-d24287829c1b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Проверен'] = df['Проверен'].astype(str)


В следующей ячейке будут выводиться все записи о действиях этого работника с указанием важной информации (на мой взгляд). Это всегда можно поменять и выводить любую другую информацию.

In [ ]:
param_zapravka = ['Дата','Топливо','Автомобиль','Заправка на стороне','Цена','Количество','Сумма']
param_ttn = ['Дата','Покупатель','Автомобиль','Количество ЗАВОД','Количество КЛИЕНТУ','Стоимость доставки']
data_driver_ttn[param_ttn]

,Дата,Покупатель,Автомобиль,Количество ЗАВОД,Количество КЛИЕНТУ,Стоимость доставки
68,2023-01-09,РСК ООО,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,9.0,9.0,5850.0
115,2023-02-09,БЕТОНКОМПЛЕКТ СПБ ООО,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,7.5,7.5,3825.0
313,2023-04-09,Митрофанов Алексей Александрович,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,8.5,9.0,5850.0
332,2023-05-09,ПРОМТЕХНИКА ООО,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,8.5,9.0,5670.0
351,2023-05-09,"ООО ""ДЕЛЬТАПРОМ""",К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,9.0,9.0,5580.0
372,2023-05-09,ПРОМТЕХНИКА ООО,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,9.0,9.0,5670.0
391,2023-05-09,ВЕКТОР ООО,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,7.0,7.0,6695.0
420,2023-06-09,МЭСС ПАО,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,9.0,9.0,4995.0
440,2023-06-09,КОРТЕКС ООО,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,8.5,8.5,4500.0
455,2023-06-09,СМУ-12 ООО,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,2.5,2.5,3850.0


In [ ]:
data_driver_zapravka[param_zapravka]

,Дата,Топливо,Автомобиль,Заправка на стороне,Цена,Количество,Сумма
235,2023-03-10,Солярка,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,Нет,59.58,80.0,4766.40
302,2023-09-28,Солярка,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,Нет,59.58,149.0,8877.42
326,2023-09-27,Солярка,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,Нет,59.58,150.0,8937.00
384,2023-09-24,Солярка,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,Нет,59.58,145.0,8639.10
410,2023-09-22,Солярка,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,Нет,59.58,148.0,8817.84
...,...,...,...,...,...,...,...
3721,2023-01-19,Солярка,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,Нет,43.49,130.0,5653.70
3728,2023-01-18,Солярка,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,Нет,43.49,146.0,6349.54
3746,2023-01-17,Солярка,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,Нет,43.49,136.0,5914.64
3764,2023-01-14,Солярка,К251РА799 Автобетоносмеситель 5814Y9 на шасси ...,Нет,56.08,147.0,8243.76


In [ ]:
car_names = data_driver_zapravka['Автомобиль'].unique()

In [ ]:
total_volume = general_parameters(data_driver_zapravka, 'Количество')
total_sum = general_parameters(data_driver_zapravka,'Сумма')
total_cost = general_parameters(data_driver_ttn, 'Стоимость доставки')
print(name)
for i in range(len(car_names)):
  volume_percar = general_parameters(split(data_driver_zapravka, 'Автомобиль', str(car_names[i])), 'Количество')
  cost_percar = general_parameters(split(data_driver_zapravka, 'Автомобиль', str(car_names[i])), 'Сумма')
  dostavka_percar = general_parameters(split(data_driver_ttn, 'Автомобиль', str(car_names[i])), 'Стоимость доставки')
  print("\033[1mДанные по машинам отдельно:\033[0m")
  print('\n')
  print("Машина работника:", car_names[i])
  print('\n')
  print( "потраченный объем топлива за период (л):", volume_percar)
  print( "потраченно денег на топливо за период (рубли):", cost_percar)
  print( "сделано доставок на стоимость за период (рубли):", dostavka_percar)
  print('\n')

print("\033[1mСуммарные данные по водителю по всем машинам:\033[0m")
print('\n')
print("объем за заданный за период (л):", total_volume)
print("стоимость топлива за период (рубли)",total_sum)
print("стоимость доставки за период (рубли):",total_cost)

print('\n')
print('\n')

Халимов Феруз Косимович
Данные по машинам отдельно:


Машина работника: К251РА799 Автобетоносмеситель 5814Y9 на шасси КАМАЗ 6540-А5


потраченный объем топлива за период (л): 15080.0
потраченно денег на топливо за период (рубли): 720697.65
сделано доставок на стоимость за период (рубли): 176187.5


Суммарные данные по водителю по всем машинам:


объем за заданный за период (л): 15080.0
стоимость топлива за период (рубли) 720697.65
стоимость доставки за период (рубли): 176187.5






<ipython-input-211-43a7e2ac14fd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[parameter] = df[parameter].astype(float)
<ipython-input-260-b4bd58b8a48e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].astype(str)
